## *Spotify recommender end-to-end*

![diagrama de bloques](./custom_spotify_recommender.png)

### 1. Recolección de datos con la API de Spotify

#### Configuración de la API

In [1]:
# Librerías requeridas
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

Inicialmente se debe configurar una aplicación en el *dashboard* de Spotify, según el [Web API tutorial](https://developer.spotify.com/documentation/web-api/quick-start/):

1. Ir a [developer.spotify.com](https://developer.spotify.com/)
2. Ir a la pestaña *DASHBOARD*
3. Ingresar con el nombre de usuario y contraseña de Spotify
4. Crear una nueva aplicación (*CREATE AN APP*), definiendo un nombre y agregando una descripción
5. *EDIT SETTINGS* -> Redirect URIs -> http://localhost:8080. Requerido para la autenticación desde Python
6. *USERS AND ACCESS* -> *ADD NEW USER* -> Definir nombre y correo electrónico de la cuenta de Spotify. Este será el usuario que podrá acceder a la API desde Python


Con la aplicación creada y configurada, volver al *DASHBOARD* y copiar los campos *Client ID* (el identificador de la aplicación) y *Client Secret* (la clave que permite acceder a la aplicación), además del nombre de usuario con acceso autorizado:

In [2]:
cid = 'Client ID obtenido en la API de Spotify'
secret = 'Client secret obtenido en la API de Spotify'
username = 'Nombre de Usuario en Spotify'
redirect_uri = 'http://localhost:8080'

#### Conexión spotipy-app

In [3]:
# Privilegios: 'user-top-read' para current_user_top_tracks;
scope = 'user-top-read'

token = util.prompt_for_user_token(username,scope,cid,secret,redirect_uri)
sp = spotipy.Spotify(auth=token)

#### Listado de top-20 tracks                              

Como punto de partida se usarán los [user's top tracks](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-users-top-artists-and-tracks) que requiere privilegios tipo `user-top-read`.

Otros puntos de partida sugeridos:
- `sp.user_current_followed_artists`
- `sp.current_user_playlists`
- `sp.current_user_recently_played`
- `sp.current_user_top_artists`

entre otros.

In [4]:
top20 = sp.current_user_top_tracks(time_range='short_term', limit=20)

In [5]:
top20

{'items': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'},
      'href': 'https://api.spotify.com/v1/artists/0epOFNiUfyON9EYx7Tpr6V',
      'id': '0epOFNiUfyON9EYx7Tpr6V',
      'name': 'The Strokes',
      'type': 'artist',
      'uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MT',
     'MX',
     'MY',
     'NI',
     'NL',
     'NO',
  

In [6]:
# El resultado está almacenado en un diccionario en donde 'items'
# contiene la información de los tracks

for i, item in enumerate(top20['items']):
    print(i+1, item['name'], '//', item['artists'][0]['name'])

1 The Adults Are Talking // The Strokes
2 ZITTI E BUONI // Måneskin
3 Chosen // Måneskin
4 Beggin' // Måneskin
5 Dani California // Red Hot Chili Peppers
6 You Should Be Dancing // Dee Gees
7 You Can't Put Your Arms Round a Memory // Billie Joe Armstrong
8 Selfless // The Strokes
9 FOR YOUR LOVE // Måneskin
10 The Getaway // Red Hot Chili Peppers
11 Vaccine // Logic
12 I WANNA BE YOUR SLAVE // Måneskin
13 Bad Decisions // The Strokes
14 Basket Case // Green Day
15 VENT'ANNI // Måneskin
16 CORALINE // Måneskin
17 Heroes - 2017 Remaster // David Bowie
18 Paper Cages // Franz Ferdinand
19 Snow (Hey Oh) // Red Hot Chili Peppers
20 Dark Necessities // Red Hot Chili Peppers


#### Dataset del preferencias del usuario

Con estos tracks crearemos un dataset (en Pandas) que contendrá, por cada pista, su identificador y sus [características sonoras]():

In [7]:
import pandas as pd

# Extraer ids y nombres de las canciones, y extraer audio_features
tracks = top20['items']
track_ids = []
track_names = []
features = []

for track in tracks:
    track_id = track['id']
    track_name = track['name']
    audio_features = sp.audio_features(track_id)
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])
    
top20_df = pd.DataFrame(features,index = track_names)

In [8]:
top20_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
The Adults Are Talking,0.593,0.749,5,-5.670,1,0.0475,0.01130,0.106000,0.3140,0.649,164.959,audio_features,5ruzrDWcT0vuJIOMW7gMnW,spotify:track:5ruzrDWcT0vuJIOMW7gMnW,https://api.spotify.com/v1/tracks/5ruzrDWcT0vu...,https://api.spotify.com/v1/audio-analysis/5ruz...,309053,4
ZITTI E BUONI,0.625,0.939,4,-3.115,0,0.0669,0.00138,0.000000,0.4240,0.644,102.999,audio_features,776AftMmFFAWUIEAb3lHhw,spotify:track:776AftMmFFAWUIEAb3lHhw,https://api.spotify.com/v1/tracks/776AftMmFFAW...,https://api.spotify.com/v1/audio-analysis/776A...,194787,4
Chosen,0.658,0.892,6,-2.928,0,0.0360,0.00723,0.000000,0.0723,0.919,107.944,audio_features,2lDgdWIGLYuk74pDeRPgPF,spotify:track:2lDgdWIGLYuk74pDeRPgPF,https://api.spotify.com/v1/tracks/2lDgdWIGLYuk...,https://api.spotify.com/v1/audio-analysis/2lDg...,162013,4
Beggin',0.714,0.800,11,-4.808,0,0.0504,0.12700,0.000000,0.3590,0.589,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
Dani California,0.556,0.913,0,-2.360,1,0.0437,0.01930,0.000009,0.3460,0.730,96.184,audio_features,10Nmj3JCNoMeBQ87uw5j8k,spotify:track:10Nmj3JCNoMeBQ87uw5j8k,https://api.spotify.com/v1/tracks/10Nmj3JCNoMe...,https://api.spotify.com/v1/audio-analysis/10Nm...,282160,4


In [9]:
# Reorganizar columnas
top20_df = top20_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [10]:
top20_df.shape

(20, 13)

#### Tracks candidatas

Partiendo de los top-20 tracks:
1. Extraeremos los artistas correspondientes (sin repeticiones)
2. Se usará la API para ampliar este listado a:
    - Artistas relacionados (`sp.artist_related_artists`)
    - Artistas con nuevos lanzamientos (`sp.new_releases`) para agregar "novedad" al playlist generado

Con este listado más amplio de artistas, obtenido en (1) y (2) se obtendrá:

3. El listado de albums
4. Y por cada album el listado de tracks

Y finalmente, para el listado resultante se obtendrán las *audio features*. El dataset resultante serán las pistas candidatas, que luego se llevarán al sistema de recomendación

In [11]:
# 1. Extraer los artistas correspondientes a las top-20 tracks (sin repeticiones)
ids_artists = []
print('Artistas en mi top20:')
print('=====================')
for item in top20['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas en mi top20:
0epOFNiUfyON9EYx7Tpr6V: The Strokes
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0L8ExT028jH3ddEcZwqJJ5: Red Hot Chili Peppers
0mCTPQ5oa1lbPvbw4kc0eX: Dee Gees
1MrEurzLcL8ugfP1PrUPWG: Billie Joe Armstrong
0epOFNiUfyON9EYx7Tpr6V: The Strokes
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0L8ExT028jH3ddEcZwqJJ5: Red Hot Chili Peppers
4xRYI6VqpkE3UwrDrAZL8L: Logic
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0epOFNiUfyON9EYx7Tpr6V: The Strokes
7oPftvlwr6VrsViSDV7fJY: Green Day
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0lAWpj5szCSwM4rUMHYmrr: Måneskin
0oSGxfWSnnOXhD2fKuz2Gy: David Bowie
0XNa1vTidXlvJ2gHSsRi4A: Franz Ferdinand
0L8ExT028jH3ddEcZwqJJ5: Red Hot Chili Peppers
0L8ExT028jH3ddEcZwqJJ5: Red Hot Chili Peppers
Número de artistas (sin repeticiones): 9


In [12]:
# 2.1 Ampliar al listado anterior a artistas relacionados
# Por cada artista que sigo y que está en mi top20 buscar artistas similares y añadirlos al listado
print('')
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id)

ids_artists.extend(ids_similar_artists)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas similares:
5iot8OPcosJN9nCl7I5SdK: Irama
3hN3iJMbbBmqBSAMx5veDa: Ultimo
2nftqfbLohpDYzY8VUlvbm: Benji & Fede
76UCIJTB0jcJvBaL0CdIqx: Takagi & Ketra
1CF7hrTuWgErEa6HBFJ8d3: Michele Bravi
2iK8weavvfS2xJCmzNzNE5: J-AX
3o7fC2O4nraaicpID6bBZW: Elettra Lamborghini
1xqolkIzTFMmqgCuD48WNt: Shade
0tTS475qIqv3KXYZMXjsYy: Lorenzo Fragola
4XWTdNlsP8jqo5BDn5hgmd: Ermal Meta
396Jr76018oUMR6QBnqT8T: Cesare Cremonini
4jFlmD92WULLlaRS8Cj6QS: Francesca Michielin
3xGlLcG9CUrs5MvFkSLOS5: Marco Mengoni
29nrwultxQ8jENVmXoyMqK: Carl Brave
3pgCLfNbw5ozIfoNsvDU7i: Fedez
0EqkKYDK9EkKY5N7zU3FPv: Annalisa
35orQw8LgQn7KOFjzCyY7E: Fabio Rovazzi
6RdcIWVKYYzNzjQRd3oyHS: Pinguini Tattici Nucleari
7KFOc3T4Xo8DVZt4PWw2qN: Gazzelle
1X9iZlQXfAAx4Vvmlqeao7: Negramaro
3IstlZaHyUP9SONpulb4SM: Chris Webby
5G9kmDLg3OeUyj8KVBLzbu: mike.
00Z3UDoAQwzvGu13HoAM7J: Skizzy Mars
382aq8Pij5V2nE2JMHMoxl: Hoodie Allen
5Z5jUyiNvFaqp0EVyLNf0p: Futuristic
02kJSzxNuaWGqwubyUba0Z: G-Eazy
1LrWZc2qPhRCHyr6XtpBxq: Dizzy Wright
7EWU4FhU

In [13]:
# 2.2 Ampliar el listado anterior con artistas con nuevos lanzamientos

print('')
print('Artistas con nuevos lanzamientos:')
print('=====================')
new_releases = sp.new_releases(limit=20)['albums']
for item in new_releases['items']:
    artist_id = item['artists'][0]['id']   #[0] porque puede haber varios artistas, se tomará el primero
    artist_name = item['artists'][0]['name']
    album_name = item['name']   # Nombre del album, puramente informativo
    release_date = item['release_date'] # Fecha de lanzamiento, puramente informativo
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas con nuevos lanzamientos:
6qqNVTkY8uBg9cP3Jd7DAH: Billie Eilish - // Happier Than Ever, 2021-07-30
0du5cEVh5yTK9QJze8zA0C: Bruno Mars - // Skate, 2021-07-30
2gqMBdyddvN82dzZt4ZF14: Yola - // Stand For Myself, 2021-07-30
4xRYI6VqpkE3UwrDrAZL8L: Logic - // Bobby Tarantino III, 2021-07-30
2eam0iDomRHGBypaDQLwWI: Bleachers - // Take the Sadness Out of Saturday Night, 2021-07-30
2tIP7SsRs7vjIcLrU85W8J: The Kid LAROI - // F*CK LOVE 3+: OVER YOU, 2021-07-27
6EPlBSH2RSiettczlz7ihV: Sleepy Hallow - // 2055 (feat. Coi Leray), 2021-07-28
5KNNVgR6LBIABRIomyCwKJ: Dermot Kennedy - // Better Days, 2021-07-28
1SupJlEpv7RS2tPNRaHViT: Nicky Jam - // Miami, 2021-07-29
4TshyQDihSYXSWqvclXl3I: Parmalee - // For You, 2021-07-30
0EFisYRi20PTADoJrifHrz: Jhay Cortez - // En Mi Cuarto, 2021-07-30
7oPxPZSk7y5q0fhzpmX5Gi: SEB - // IT’S OKAY, WE’RE DREAMING, 2021-07-30
1l2ekx5skC4gJH8djERwh1: Don Diablo - // Tears For Later, 2021-07-30
5a2EaR3hamoenG9rDuVn8j: Prince - // Welcome 2 America, 2021-07-30
46gy

In [14]:
# Obtener el listado de albums de cada uno de los anteriores artistas.
# Se limitará a únicamente 1 album (limit=1), para evitar tener un listado gigantesco

id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artista {i+1} de {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1) # para evitar tener una lista gigantesca
    for album in albums['items']:
        id_albums.append(album['id'])
print('¡Listo!')

Procesando artista 1 de 183...
Procesando artista 2 de 183...
Procesando artista 3 de 183...
Procesando artista 4 de 183...
Procesando artista 5 de 183...
Procesando artista 6 de 183...
Procesando artista 7 de 183...
Procesando artista 8 de 183...
Procesando artista 9 de 183...
Procesando artista 10 de 183...
Procesando artista 11 de 183...
Procesando artista 12 de 183...
Procesando artista 13 de 183...
Procesando artista 14 de 183...
Procesando artista 15 de 183...
Procesando artista 16 de 183...
Procesando artista 17 de 183...
Procesando artista 18 de 183...
Procesando artista 19 de 183...
Procesando artista 20 de 183...
Procesando artista 21 de 183...
Procesando artista 22 de 183...
Procesando artista 23 de 183...
Procesando artista 24 de 183...
Procesando artista 25 de 183...
Procesando artista 26 de 183...
Procesando artista 27 de 183...
Procesando artista 28 de 183...
Procesando artista 29 de 183...
Procesando artista 30 de 183...
Procesando artista 31 de 183...
Procesando artist

In [15]:
# Por cada album extraer 3 tracks

id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    print(f'Procesando album {i+1} de {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=3)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
print(f'¡Listo! Número total de tracks pre-candidatos: {len(id_tracks)}')

Procesando album 1 de 183...
Procesando album 2 de 183...
Procesando album 3 de 183...
Procesando album 4 de 183...
Procesando album 5 de 183...
Procesando album 6 de 183...
Procesando album 7 de 183...
Procesando album 8 de 183...
Procesando album 9 de 183...
Procesando album 10 de 183...
Procesando album 11 de 183...
Procesando album 12 de 183...
Procesando album 13 de 183...
Procesando album 14 de 183...
Procesando album 15 de 183...
Procesando album 16 de 183...
Procesando album 17 de 183...
Procesando album 18 de 183...
Procesando album 19 de 183...
Procesando album 20 de 183...
Procesando album 21 de 183...
Procesando album 22 de 183...
Procesando album 23 de 183...
Procesando album 24 de 183...
Procesando album 25 de 183...
Procesando album 26 de 183...
Procesando album 27 de 183...
Procesando album 28 de 183...
Procesando album 29 de 183...
Procesando album 30 de 183...
Procesando album 31 de 183...
Procesando album 32 de 183...
Procesando album 33 de 183...
Procesando album 34

In [16]:
# En total se tendrán 543 pistas candidatas. Para cada una se extraerán los
# audio_features y todo se almacenará en un dataframe

track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Procesando track {i+1} de {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    #No incluir pistas sin "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])
print('¡Listo!')

candidatos_df = pd.DataFrame(features,index = track_names)

Procesando track 1 de 543...
Procesando track 2 de 543...
Procesando track 3 de 543...
Procesando track 4 de 543...
Procesando track 5 de 543...
Procesando track 6 de 543...
Procesando track 7 de 543...
Procesando track 8 de 543...
Procesando track 9 de 543...
Procesando track 10 de 543...
Procesando track 11 de 543...
Procesando track 12 de 543...
Procesando track 13 de 543...
Procesando track 14 de 543...
Procesando track 15 de 543...
Procesando track 16 de 543...
Procesando track 17 de 543...
Procesando track 18 de 543...
Procesando track 19 de 543...
Procesando track 20 de 543...
Procesando track 21 de 543...
Procesando track 22 de 543...
Procesando track 23 de 543...
Procesando track 24 de 543...
Procesando track 25 de 543...
Procesando track 26 de 543...
Procesando track 27 de 543...
Procesando track 28 de 543...
Procesando track 29 de 543...
Procesando track 30 de 543...
Procesando track 31 de 543...
Procesando track 32 de 543...
Procesando track 33 de 543...
Procesando track 34

Procesando track 269 de 543...
Procesando track 270 de 543...
Procesando track 271 de 543...
Procesando track 272 de 543...
Procesando track 273 de 543...
Procesando track 274 de 543...
Procesando track 275 de 543...
Procesando track 276 de 543...
Procesando track 277 de 543...
Procesando track 278 de 543...
Procesando track 279 de 543...
Procesando track 280 de 543...
Procesando track 281 de 543...
Procesando track 282 de 543...
Procesando track 283 de 543...
Procesando track 284 de 543...
Procesando track 285 de 543...
Procesando track 286 de 543...
Procesando track 287 de 543...
Procesando track 288 de 543...
Procesando track 289 de 543...
Procesando track 290 de 543...
Procesando track 291 de 543...
Procesando track 292 de 543...
Procesando track 293 de 543...
Procesando track 294 de 543...
Procesando track 295 de 543...
Procesando track 296 de 543...
Procesando track 297 de 543...
Procesando track 298 de 543...
Procesando track 299 de 543...
Procesando track 300 de 543...
Procesan

Procesando track 534 de 543...
Procesando track 535 de 543...
Procesando track 536 de 543...
Procesando track 537 de 543...
Procesando track 538 de 543...
Procesando track 539 de 543...
Procesando track 540 de 543...
Procesando track 541 de 543...
Procesando track 542 de 543...
Procesando track 543 de 543...
¡Listo!


In [17]:
candidatos_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Generation Rx,0.428,0.335,6,-14.428,0,0.0354,0.851000,0.357000,0.2240,0.0396,125.083,audio_features,7xAqA8s3fhOEu4yLgYJ1q2,spotify:track:7xAqA8s3fhOEu4yLgYJ1q2,https://api.spotify.com/v1/tracks/7xAqA8s3fhOE...,https://api.spotify.com/v1/audio-analysis/7xAq...,127393,4
Self Help,0.360,0.972,6,-3.574,0,0.1250,0.000207,0.000123,0.3440,0.2780,174.963,audio_features,33H1pDEn8hdNGDetae2w0K,spotify:track:33H1pDEn8hdNGDetae2w0K,https://api.spotify.com/v1/tracks/33H1pDEn8hdN...,https://api.spotify.com/v1/audio-analysis/33H1...,203648,4
Shadowboxer,0.401,0.944,6,-5.008,0,0.0632,0.000284,0.000077,0.0747,0.4160,173.074,audio_features,3yNVDeOlaPGu2Ab71gG5nP,spotify:track:3yNVDeOlaPGu2Ab71gG5nP,https://api.spotify.com/v1/tracks/3yNVDeOlaPGu...,https://api.spotify.com/v1/audio-analysis/3yNV...,185013,4
Nothing Breaks Like A Heart - Don Diablo Remix,0.712,0.871,7,-3.571,0,0.2240,0.179000,0.000002,0.2030,0.3790,123.975,audio_features,29fPhldPheJlrr55sVHPqo,spotify:track:29fPhldPheJlrr55sVHPqo,https://api.spotify.com/v1/tracks/29fPhldPheJl...,https://api.spotify.com/v1/audio-analysis/29fP...,216774,4
Levitating - Don Diablo Remix,0.683,0.918,11,-3.029,0,0.1550,0.103000,0.000001,0.4710,0.3720,119.972,audio_features,4zR4PEQl6wQlEP7IhsaTjo,spotify:track:4zR4PEQl6wQlEP7IhsaTjo,https://api.spotify.com/v1/tracks/4zR4PEQl6wQl...,https://api.spotify.com/v1/audio-analysis/4zR4...,208000,4


In [18]:
candidatos_df = candidatos_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [19]:
candidatos_df.shape

(543, 13)

In [20]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Generation Rx,7xAqA8s3fhOEu4yLgYJ1q2,0.851000,0.428,127393,0.335,0.357000,6,0.2240,-14.428,0,0.0354,125.083,0.0396
Self Help,33H1pDEn8hdNGDetae2w0K,0.000207,0.360,203648,0.972,0.000123,6,0.3440,-3.574,0,0.1250,174.963,0.2780
Shadowboxer,3yNVDeOlaPGu2Ab71gG5nP,0.000284,0.401,185013,0.944,0.000077,6,0.0747,-5.008,0,0.0632,173.074,0.4160
Nothing Breaks Like A Heart - Don Diablo Remix,29fPhldPheJlrr55sVHPqo,0.179000,0.712,216774,0.871,0.000002,7,0.2030,-3.571,0,0.2240,123.975,0.3790
Levitating - Don Diablo Remix,4zR4PEQl6wQlEP7IhsaTjo,0.103000,0.683,208000,0.918,0.000001,11,0.4710,-3.029,0,0.1550,119.972,0.3720


In [21]:
top20_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
The Adults Are Talking,5ruzrDWcT0vuJIOMW7gMnW,0.01130,0.593,309053,0.749,0.106000,5,0.3140,-5.670,1,0.0475,164.959,0.649
ZITTI E BUONI,776AftMmFFAWUIEAb3lHhw,0.00138,0.625,194787,0.939,0.000000,4,0.4240,-3.115,0,0.0669,102.999,0.644
Chosen,2lDgdWIGLYuk74pDeRPgPF,0.00723,0.658,162013,0.892,0.000000,6,0.0723,-2.928,0,0.0360,107.944,0.919
Beggin',3Wrjm47oTz2sjIgck11l5e,0.12700,0.714,211560,0.800,0.000000,11,0.3590,-4.808,0,0.0504,134.002,0.589
Dani California,10Nmj3JCNoMeBQ87uw5j8k,0.01930,0.556,282160,0.913,0.000009,0,0.3460,-2.360,1,0.0437,96.184,0.730


### 2. Sistema de recomendación

![el sistema de recomendación](./sistema_recomendacion.png)

Veamos en detalle el sistema de recomendación:

![el sistema de recomendación en detalle](./sistema_recomendacion_detalle.png)


#### El filtrado basado en contenido

Permite cuantificar qué tan similar es un ítem de `candidatos_df` a un ítem de `top20_df`.

Una forma de hacer esta comparación es usando la similitud del coseno:

![vectores de características](./vectores_caracteristicas.png)

![la similitud del coseno](./similitud_coseno.png)

Calcularemos este similitud entre cada top-20 y cada una de las pistas candidatas (matriz de 20 x n_pistas_candidatas)

In [22]:
# Extraer sólo los features en formato numpy array
top20_mtx = top20_df.iloc[:,1:].values
candidatos_mtx = candidatos_df.iloc[:,1:].values

In [23]:
from sklearn.preprocessing import StandardScaler

# Estandarizar cada columna de features: mu = 0, sigma = 1
# pues cada característica tiene una escala diferente
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

In [24]:
print(t20_scaled.mean(axis=0))
print(t20_scaled.std(axis=0))

[-1.30451205e-16 -4.49640325e-16  1.05471187e-16  5.80091530e-16
  2.77555756e-18  5.55111512e-18  6.10622664e-17  4.44089210e-17
  4.44089210e-17  2.77555756e-18  3.33066907e-16  6.32827124e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [25]:
import numpy as np

# Normalizar cada vector de características (es decir por filas)

# Magnitudes de cada vector (o pista)
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))

# Normalización
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

print(np.sqrt((t20*t20).sum(axis=1)))
print(np.sqrt((can*can).sum(axis=1)))


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [26]:
from sklearn.metrics.pairwise import linear_kernel

# Calcular similitudes del coseno entre cada top-20 y cada
# una de las pistas candidatas
cos_sim = linear_kernel(t20,can)
cos_sim.shape

(20, 543)

In [27]:
# Ejemplo: ¿qué tanto se parece una pista candidata a una del top-20?
print(cos_sim[6,270])
print(cos_sim[3,24])


0.5215138618987606
0.007047208799871543


In [28]:
# Obtener candidatos para una pista dada

# Dada una pista del top-20 (pos = 0, 1, ..., 19) extraer "ncands" candidatos, usando
# "cos_sim" y siempre y cuando superen un umbral de similitud

def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.8):
    # Obtener todas las pistas candidatas por encima de umbral
    idx = np.where(cos_sim[pos,:]>=umbral)[0] # ejm. idx: [27, 82, 135]
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncands", retornar únicamente un total de "ncands"
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [29]:
# Ejemplo de uso
for i in range(20):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i}==> pistas candidatas: {cands}, similitudes: {cos_sim[i,cands]}')

0==> pistas candidatas: [], similitudes: []
1==> pistas candidatas: [ 71  78 334], similitudes: [0.89542617 0.85824661 0.80710631]
2==> pistas candidatas: [58], similitudes: [0.85491718]
3==> pistas candidatas: [], similitudes: []
4==> pistas candidatas: [306 503 251  40  17], similitudes: [0.82675576 0.82611856 0.81411    0.8088681  0.80771745]
5==> pistas candidatas: [42], similitudes: [0.87227163]
6==> pistas candidatas: [], similitudes: []
7==> pistas candidatas: [126 274  68 389], similitudes: [0.92052708 0.86319636 0.84764199 0.80897689]
8==> pistas candidatas: [ 71 305  78 334  64], similitudes: [0.94449459 0.921214   0.89761485 0.84612035 0.82271043]
9==> pistas candidatas: [], similitudes: []
10==> pistas candidatas: [397 158], similitudes: [0.92477213 0.82660251]
11==> pistas candidatas: [364 158  92 397 262], similitudes: [0.87946925 0.83831387 0.8354714  0.82139384 0.80957556]
12==> pistas candidatas: [200 266 209 487], similitudes: [0.89268919 0.88680876 0.83209262 0.82803

#### Creación de playlist con listado de pistas sugeridas

In [30]:
# Para crear la playlist se requieren únicamente los ids
ids_t20 = []
ids_playlist = []

for i in range(top20_df.shape[0]):
    print(top20_df.index[i])   # Nombre de la pista en el top-20
    ids_t20.append(top20_df['id'][i])
    
    # Obtener listado de candidatos para esta pista
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    # Si hay pistas relacionadas obtener los ids correspondientes
    # e imprimir en pantalla
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Obtener los ids correspondientes e imprimir en pantalla
        for j in cands:
            id_cand = candidatos_df['id'][j]
            ids_playlist.append(id_cand)
            
            # E imprimir en pantalla el candidato
            print(f'   {candidatos_df.index[j]}')

The Adults Are Talking
     ***No se encontraron pistas relacionadas***
ZITTI E BUONI
   More-More-More
   ZITTI E BUONI
   20/20 Vision
Chosen
   Oblivion
Beggin'
     ***No se encontraron pistas relacionadas***
Dani California
   Hippy's Son
   Telegraph Avenue
   Delivery
   Don't Look At Me That Way
   Sad Sad City
You Should Be Dancing
   About A Girl - Live Version
You Can't Put Your Arms Round a Memory
     ***No se encontraron pistas relacionadas***
Selfless
   I'd Have You Anytime - 2020 Mix
   Calci e pugni
   Mantieni il bacio
   Un milione di cose da dirti
FOR YOUR LOVE
   More-More-More
   Buzzards And Crows
   ZITTI E BUONI
   20/20 Vision
   A Thousand Words
The Getaway
     ***No se encontraron pistas relacionadas***
Vaccine
   Vaccine
   Leave Me Alone
I WANNA BE YOUR SLAVE
   Last Day
   Leave Me Alone
   Bend
   Vaccine
   Love → Building on Fire - 2003 Remaster
Bad Decisions
   Slow Poison
   The Wolfpack
   This Is How I Disappear
   Bored and Razed - Recorded at E

In [31]:
# Eliminar candidatos que ya están en el top-20
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]

# Y eliminar posibles repeticiones
ids_playlist_dep = list(set(ids_playlist_dep))

In [32]:
print(ids_playlist_dep)

['70ezKh3AXSA29EXbYZXvHM', '4eAy0nOh3g1fwGAb2OVQtD', '6PEQCw5UdjZDpsOxVV8AuG', '2Ye7dgHsrrb8Dw2d9SKHGJ', '3Ye5icBka8ODjcaEQakPvZ', '31RJ1xFMQCGfGniKY4IMdO', '0lgiMprywlwW4XlzrnZHCA', '3W3WNGmdYd0XkIDnCUA9P8', '52a1wWAZ206Ptr50JHaoex', '15m0MEyKTpuwwdEBGAghyL', '4t1rSpSMbGuiWUokubPtes', '3MFOpHTQc46TdIIILH9dAd', '51KMpn0p5Vb7XVdV7mWau5', '4joEqwMI7bdqunkVZ6zTqU', '5DlfBSLJiuoiWbP5WKocMI', '3MY5hKy9nT2D1gEdg3UFVv', '6lMPUipRF49L8AxLW3F40B', '6QonLH4JR5jpbLRKGAiTXc', '3dExW6DT0o338hTZA74hAl', '5IU4Ym8qNpQFIo0EXUceRr', '4VwXf1G5Is02g6pgBbYw9s', '1cnBCwceV5NhMkpzc9sdDF', '5hpyKjTtr5HsvdgLatWzJt', '3s9febMNqBjDT0o0q5hbWt', '507Z8HdRGcRExCKrc3jcp2', '5Ti7Suj14N5hmMOWdjpDrA', '3TtDUP3ijbAmWLmDAyrBe1', '16muBPcWmkHSl1uM4tE8WL', '0a70Tloqeldzv0Hp9CvgeT', '0SlVngDKKsbGjo1V0mOtG7', '7vBczWEtVRGXkI5HOISv7L', '77NmNVQxwbKwE1HqptcAKi', '3wKSnl3YTPTcyHY2hSxfde', '3oF1sqZpoarrfWGk2CGq19', '6Jg5VEDvoEtwgM2ZfkI2wS']


In [33]:
len(ids_playlist_dep) # Hemos pasado de ~500 pre-candidatos a muchas menos pistas

35

In [34]:
# Crear el playlist en spotify
pl = sp.user_playlist_create(user = username, 
                            name = 'Spotipy Recommender Playlist',
                            description = 'Playlist creada con el sistema de recomendación')

sp.playlist_add_items(pl['id'],ids_playlist_dep)

{'snapshot_id': 'MywzMzZmNmUzNzIzOTJhMTZmYTk1MGU2NzdiOTY4NTNiZGM3ZGU5YmIw'}